In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
bc = datasets.load_breast_cancer()

X,y = bc.data, bc.target

n_samples, n_features = X.shape

n_samples, n_features

(569, 30)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1234)


In [16]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [7]:
class Regression(nn.Module):
    def __init__(self, n_input_features):
        super(Regression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
    
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [5]:
model = Regression(n_features)

criterion = nn.BCELoss()

learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
epochs = 100

for epoch in range(epochs):
    y_pred = model(X_train)
    
    loss = criterion(y_pred, y_train)
    
    loss.backward()
    
    optimizer.step()
    
    optimizer.zero_grad()
    
    if epoch % 9 == 0:
        print(f"epoch:{epoch+1}, loss={loss.item():.4f}")

        


epoch:1, loss=0.7460
epoch:10, loss=0.5967
epoch:19, loss=0.5028
epoch:28, loss=0.4391
epoch:37, loss=0.3930
epoch:46, loss=0.3582
epoch:55, loss=0.3308
epoch:64, loss=0.3086
epoch:73, loss=0.2903
epoch:82, loss=0.2748
epoch:91, loss=0.2616
epoch:100, loss=0.2501


In [9]:
with torch.no_grad():
    prediction = model(X_test)
    prediction = prediction.round()
    acc = prediction.eq(y_test).sum() / float(y_test.shape[0])
    
    print(f"accuracy= {acc:.4f}")

accuracy= 0.9035
